
## Understanding Pawpular Model


### Pawpular model was first written by Abhisek Thakur.
### Later I copied the model from Manyu Li (who orginally got it from Chris Deotte)

### Link: https://www.kaggle.com/manyuli/rapids-svr-boost-17-8-pro?scriptVersionId=77613696

## Goal of this notebook: explain the Pawpular model code

## Import Libraries

In [ ]:
# !pip install tez
# !pip install timm

In [ ]:
# based on the post here: https://www.kaggle.com/c/petfinder-pawpularity-score/discussion/275094

import sys
sys.path.append("../input/tez-lib/")
sys.path.append("../input/timmmaster/")

import tez
import albumentations
import pandas as pd
import cv2
import numpy as np
import timm
import torch.nn as nn
from sklearn import metrics
import torch
from tez.callbacks import EarlyStopping
from tqdm import tqdm
import math

class args:
    batch_size = 16
    image_size = 384
    
def sigmoid(x):
    return 1 / (1 + math.exp(-x))

## Pawpular Model

In [ ]:
class PawpularDataset:
    def __init__(self, image_paths, dense_features, targets, augmentations):
        self.image_paths = image_paths
        self.dense_features = dense_features
        self.targets = targets
        self.augmentations = augmentations
        
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, item):
        image = cv2.imread(self.image_paths[item])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        if self.augmentations is not None:
            augmented = self.augmentations(image=image)
            image = augmented["image"]
            
        image = np.transpose(image, (2, 0, 1)).astype(np.float32)
        
        features = self.dense_features[item, :]
        targets = self.targets[item]
        
        return {
            "image": torch.tensor(image, dtype=torch.float),
            "features": torch.tensor(features, dtype=torch.float),
            "targets": torch.tensor(targets, dtype=torch.float),
        }
    
class PawpularModel(tez.Model):
    def __init__(self, model_name):
        super().__init__()
        self.model = timm.create_model(model_name, pretrained=False, in_chans=3)
        self.model.head = nn.Linear(self.model.head.in_features, 128)
        self.dropout = nn.Dropout(0.1)
        self.dense1 = nn.Linear(140, 64)
        self.dense2 = nn.Linear(64, 1)

    def forward(self, image, features, targets=None):
        x1 = self.model(image)
        x = self.dropout(x1)
        x = torch.cat([x, features], dim=1)
        x = self.dense1(x)
        x = self.dense2(x)
        
        x = torch.cat([x, x1, features], dim=1)
        return x, 0, {}
    

test_aug = albumentations.Compose(
    [
        albumentations.Resize(args.image_size, args.image_size, p=1),
        albumentations.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05, rotate_limit=15, p=0.5),
        albumentations.RGBShift(r_shift_limit=15, g_shift_limit=15, b_shift_limit=15, p=0.5),
        albumentations.RandomBrightnessContrast(p=0.5),
        
        albumentations.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225],
            max_pixel_value=255.0,
            p=1.0,
        ),
    ],
    p=1.0,
)

# Line by line Pawpular model

In [ ]:
# Line 1
class PawpularModel(tez.Model):

## The first line we are defining a model class and notice instead of PawpularModel(), we are using PawpularModel(tez.Model). 
## This is because we want to inherit from tez.Model instead of nn.Module.
## Note that nn.Module is base class for all neural network modules.

In [ ]:
# Line 2 and 3
def __init__(self, model_name):
        super().__init__()

## The 2nd line - "\_\_init\_\_" is a reserved method in Python classes. The word 'self' is used to represent the instance of a class.
## The 3rd line - super() has two use cases:
### 1. allows us to avoid using base class explicitly 2. working with multiple inheritance (check reference 4 for examples)

In [ ]:
# Line 4
self.model = timm.create_model(model_name, pretrained=False, in_chans=3)

## The 4th line uses timm.create_model - \'timm\' is a deep-learning library created by Ross Wightman.
## Quote from documentation - \" The create_model function is what is used to create hundreds of models inside timm. It also expects a bunch of **kwargs such as features_only and out_indices and passing these two **kwargs to the create_model function creates a feature extractor  instead.\"

## You might be wondering what's \" in_chans = 3\" - this means 3- channel images or RGB. Each pixel is made up of three channels, with each channel representing a colour.

In [ ]:
# Line 5
 self.model.head = nn.Linear(self.model.head.in_features, 128)

## What is nn.Linear? More importantly why self.model.head? 
## nn.Linear applies a linear transformation to the incoming data: $$ y = xA^T + b $$

## Let's understand self.model.head by adding print statements before and after
## The output from print statements will be
## Linear(in_features=1536, out_features=1000, bias=True) 
## Linear(in_features=1536, out_features=128, bias=True) 
## Essentially timm.create_model generates multiple models and \".head\" selects the first of those.

In [ ]:
# Lines 6,7, and 8
self.dropout = nn.Dropout(0.1)
self.dense1 = nn.Linear(140, 64)
self.dense2 = nn.Linear(64, 1)

## Line 6 is dropout - drops some data and is an effective technique for regularization
## Please see reference 7 for more info.
## Line 7 is the first dense layer and 140 is sum of 128 (output of previous layer) + dense features (12) (140 explanation thanks to Harshit Mehta (https://www.kaggle.com/harshit92))

In [ ]:
# forward block
def forward(self, image, features, targets=None):
    x1 = self.model(image)
    x = self.dropout(x1)
    x = torch.cat([x, features], dim=1)
    x = self.dense1(x)
    x = self.dense2(x)

    x = torch.cat([x, x1, features], dim=1)
    return x, 0, {}

## In the forward pass block, we define how data flows from one layer to another inside the network.
## torch.cat concatenates the given sequence of seq tensors in the given dimension.
## dim = 1 means by column and dim = 0 is by row

# References:
## 1. https://pytorch.org/docs/stable/generated/torch.nn.Module.html
## 2. https://github.com/abhishekkrthakur/tez
## 3. https://www.tutorialspoint.com/What-is-difference-between-self-and-init-methods-in-python-Class
## 4. https://www.programiz.com/python-programming/methods/built-in/super
## 5. https://fastai.github.io/timmdocs/
## 6. https://pytorch.org/docs/stable/generated/torch.nn.Linear.html
## 7. https://pytorch.org/docs/stable/generated/torch.nn.Dropout.html
## 8. https://pytorch.org/docs/stable/generated/torch.cat.html